In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from tqdm import tqdm_notebook
import pandas as pd
import time


In [2]:
# 브라우저 꺼짐 방지 옵션
chrome_options = Options()
chrome_options.headless=True
chrome_options.add_experimental_option("detach", True)


In [3]:
def scroll_down(driver):
    last_element = None  
    while True:
        url_raw = driver.find_elements(By.CSS_SELECTOR, 'div.sc-d609d44f-0.grDLmW a') 
        if not url_raw: 
            break
        if last_element == url_raw[-1]:  
            return url_raw
        last_element = url_raw[-1]  
        ActionChains(driver).move_to_element(last_element).perform()
        time.sleep(1)  

In [4]:
driver = webdriver.Chrome(options=chrome_options)
driver.set_window_size(1900, 1000)
# 웹페이지가 로드될 때까지 2초를 대기
driver.implicitly_wait(time_to_wait=2)  

# 원티드드 사이트로 이동합니다.
driver.get(url='https://jumpit.saramin.co.kr/positions?career=0&sort=rsp_rate')
driver.implicitly_wait(3)
develope_oc=driver.find_elements(By.CLASS_NAME, 'sc-b966f376-8.cZxZKh')
Occupation_list={"종료":0}
for i in range(len(develope_oc)):
    Occupation_list[develope_oc[i].text]=i+1
print(Occupation_list)
time.sleep(1)
while True:
    input_occupation=input('직군선택: ')
    if input_occupation == '종료':
        break
    occupation=Occupation_list[input_occupation]
    driver.find_element(By.XPATH, f'/html/body/main/div/section/section/div[1]/button[{occupation}]/span').click()
time.sleep(1)
scroll_down(driver)
url_raw=scroll_down(driver)
url_list, location_list=[],[]
for url in url_raw:
    try:
        link = url.get_attribute("href")  
        url_list.append(link)
        location = url.find_element(By.CSS_SELECTOR, "ul.sc-15ba67b8-1.cdeuol li:first-child").text
        location_list.append(location)
    except:
        pass
print(len(url_list))



{'종료': 0, '전체': 1, '서버/백엔드 개발자': 2, '프론트엔드 개발자': 3, '웹 풀스택 개발자': 4, '안드로이드 개발자': 5, 'iOS 개발자': 6, '크로스플랫폼 앱개발자': 7, '게임 클라이언트 개발자': 8, '게임 서버 개발자': 9, 'DBA': 10, '빅데이터 엔지니어': 11, '인공지능/머신러닝': 12, 'devops/시스템 엔지니어': 13, '정보보안 담당자': 14, 'QA 엔지니어': 15, '개발 PM': 16, 'HW/임베디드': 17, 'SW/솔루션': 18, '웹퍼블리셔': 19, 'VR/AR/3D': 20, '블록체인': 21, '기술지원': 22}
51


In [5]:
driver = webdriver.Chrome(options=chrome_options)
driver.set_window_size(1900, 1000)
# 웹페이지가 로드될 때까지 2초를 대기
driver.implicitly_wait(2)
data_dict={}

for n in tqdm_notebook(range(len(url_list)),desc="진행률"):
    data_info={}
    Main_task, qualifications, Preferred_Qualifications=None, None, None
    driver.get(url_list[n])
    driver.implicitly_wait(2)
    try:
        driver.find_element(By.CLASS_NAME, 'sc-52a3e76a-0 frGwWL').click()
    except:
        pass
    driver.implicitly_wait(2)
    title = driver.find_element(By.CLASS_NAME, 'sc-923c3317-0.jIWuEG h1').text
    explanation_things=driver.find_elements(By.CLASS_NAME, "sc-e76d2562-1.eALIki")
    explanation_detail=driver.find_elements(By.CSS_SELECTOR, "dl.sc-e76d2562-0.jqzywl dd pre")
    for i in range(len(explanation_things)):
        if explanation_things[i].text == '주요업무':
            Main_task=explanation_detail[i].text
        elif explanation_things[i].text == '자격요건':
            qualifications=explanation_detail[i].text
        elif explanation_things[i].text == '우대사항':
            Preferred_Qualifications=explanation_detail[i].text
    skills=[]
    try:
        skill_element = driver.find_elements(By.CSS_SELECTOR, 'div.sc-9db2de1-0.hMgXwE img')
        for skill in skill_element:
            skills.append(skill.get_attribute('alt'))
    except:
        skills=None
    
    data_info['title']=title
    data_info['location']=location_list[n]
    data_info['Main_task']=Main_task
    data_info['qualifications']=qualifications
    data_info['Preferred_Qualifications']=Preferred_Qualifications
    data_info['skills']=skills
    data_info['url']=url_list[n]
    
    data_dict[n]=data_info
    time.sleep(0.5)
    print(n,title)
    


C:\Users\User\AppData\Local\Temp\ipykernel_2944\1257182975.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for n in tqdm_notebook(range(len(url_list)),desc="진행률"):


진행률:   0%|          | 0/51 [00:00<?, ?it/s]

0 인공지능(AI) 엔지니어 신입 채용
1 ML Engineer - Speech Processing (신입)
2 [전문연구요원]AI연구엔지니어-LLM
3 딥러닝, AI(컴퓨터비전) 분야 응용 프로그래머 채용(전문연구요원)
4 Localization Engineer (Junior, 신입)
5 ML Engineer (Junior & Senior)
6 [전문연구요원]AI엔지니어-DocumentAI
7 AI(딥러닝) 연구 및 개발 채용
8 [병역특례]ML Engineer -전문연구요원
9 디지털헬스케어 플랫폼 AI 엔지니어
10 이미지 프로세싱 알고리즘 라이브러리(2D,3D,AI 딥러닝) 개발 신입 채용
11 AI/LLM Curriculum Developer
12 AI 개발자 연구원 신입
13 ADAS NPU AI SW Engineer
14 AI Research Engineer - LLM
15 AI 최적화 Researcher 채용
16 AI Research Engineer - Document AI
17 Artificial Intelligence Researcher 채용
18 Runtime 소프트웨어 엔지니어 채용
19 LLM Researcher (경력 5년이하)
20 Document AI Researcher (경력 3년이하)
21 코덱 SW 엔지니어 (알고리즘 개발/신입)
22 임베디드 AI 개발자 (C++)
23 Vision AI Application Engineer
24 임베디드 AI 개발자 (병역특례)
25 코덱SW엔지니어 (화질분석/신입)
26 Model Compression Research (병역특례)
27 Model Compression Research Engineer
28 LLM Research Engineer
29 실내 내비게이션 연구/개발(전문연구요원가능)
30 Vision AI Application Engineer(병역특례)
31 LLM Research Engineer (병역특례)
32 언어모델 딥러닝 개발자 (신입)
33 ML 엔지니어/Resear

In [6]:
import pandas as pd

csv_file = "wanted.csv"
wanted_df = pd.read_csv(csv_file)
jumpit_df = pd.DataFrame.from_dict(data_dict, 'index')
final_df = pd.concat([wanted_df, jumpit_df], ignore_index=True)
final_df.to_csv("datasets.csv", index=False)


In [7]:
len(final_df)

112

In [8]:
final_df.head()

,title,location,Main_task,qualifications,Preferred_Qualifications,skills,url
0,[신입] 머신러닝 엔지니어 (LLM),서울 서초구,• 데이터를 분석하고 풀어야 할 문제를 정의합니다.\n• 동료들과 더 나은 LLM ...,• LLM(Fine tuning 포함) 또는 NLP 모델의 학습 경험이 있는 분\n...,• 다양한 분야의 데이터를 다룬 경험이 있는 분\n• AI Agent 활용 경험이 ...,[],https://www.wanted.co.kr/wd/264181
1,[카카오계열사]Prompt Engineer,서울 강남구,"• LLM 프롬프트 최적화 및 엔지니어링: 다양한 LLM(GPT, Claude, G...",• 해외여행 결격사유가 없는 분\n• LLM 및 NLP 실무 경험: LLM 또는 자...,ML 기반 서비스 개발 또는 백엔드 개발 경험\n• ML 기반 서비스 개발 경험: ...,"['Python', 'NLP']",https://www.wanted.co.kr/wd/264121
2,Google Cloud Platform Solutions Engineer,서울 강남구,• GCP 클라우드 아키텍처 설계 및 기술 컨설팅 수행\n• 고객사 클라우드 마이그...,• 클라우드 환경에서의 프로젝트 경험 보유 (토이 프로젝트 포함)\n• Genera...,• Generative AI 프로젝트 경험자\n• AI/ML 프로젝트 수행 경험 또...,[],https://www.wanted.co.kr/wd/263995
3,코덱SW엔지니어 (화질분석/신입),서울 강남구,"[다음 업무를 통해 목표를 달성하고 성장합니다]\n• 영상처리, 동영상 압축 알고리...","[이런분과 함께하고 싶어요]\n• 전자, 컴퓨터공학 학사 이상\n• C, Pytho...",[이런분이라면 더욱 환영해요]\n• 비디오 압축 혹은 영상 처리 관련 연구 수행 및...,"['Linux', 'Java', 'Python', 'C', 'C++']",https://www.wanted.co.kr/wd/174091
4,코덱SW엔지니어 (알고리즘개발/신입),서울 강남구,"[이런분과 함께하고 싶어요]\n• 영상처리, 동영상 압축 등 멀티미디어 처리 기술개발","[이런분이라면 더욱 환영해요]\n• 전자, 컴퓨터공학 학사 이상\n• C/C++ 능...",[더 몰입하실 수 있도록 도와드려요]\n• 비디오 압축 혹은 영상 처리 관련 연구 ...,"['Linux', 'Java', 'Python', 'C', 'C++']",https://www.wanted.co.kr/wd/174088
